In [9]:
#
import requests
from bs4 import BeautifulSoup
import urllib.request
import json
import csv
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import time
from selenium.webdriver.chrome.options import Options


In [4]:
#https://www.alko.fi/en/products/934133/Zuccardi-Finca-Piedra-Infinita-Supercal-2016/
async def  getWine(url):
    url = url
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    response = requests.get(url,  headers=headers)

    # if int(response.status_code) != 200:
    #     return

    soup=BeautifulSoup(response.content, "html.parser")
    # DETAILS
    mostplayed=soup.findAll("div", {"class": "hard-facts"})
    vineStat = dict()

    for ul in mostplayed:
        for li in ul.findAll('li'):
            label = li.find("div", {"class" : "fact-label"}).text.strip()
            data = li.find("div", {"class" : "fact-data"}).text.strip().replace("\xa0"," ")
            vineStat[label] = data

    # Food pairings        
    pairingWeb=soup.findAll("ul", {"class": "food-pairings"})
    foodPairings = []
    for ul in pairingWeb:
        for a in ul.findAll('a'):
            try:
                food = json.loads(a['data-applied-filters'][1:-1])
                foodPairings.append(food['filterValue'])
                
            except:
                pass
    foodPairings = list(set(foodPairings))


    # tastes
    tastes=soup.findAll("div", {"class": "html-print"})
    text = ""
    topNotch = ""
    tips = ""
    try:
        text = tastes[3:5][1].text.strip().replace("\n", " ")
        topNotch = tastes[3:5][0].text.strip()
        tips = tastes[5:6][0].text.strip().replace("\n", " ")
    except:
        pass
    return {
        'pairings' : foodPairings,
        'stats' : vineStat,
        'text' : text,
        'topNotch' : topNotch,
        'tips' : tips

    }

In [7]:
#files = [("box", "box308"), ("desert", "desert463"), ("red", "red2955"), ("rose", "rose364"), ("sparkling", "sparkling1281"), ("white", "white2086")]
files = [("rose", "rose364"),("red", "red2955"),  ("sparkling", "sparkling1281"), ("white", "white2086")]

for wine in files:
    print("Starting wine "+wine[0])
    with open('dataset_'+wine[0]+'.csv', 'w', newline='',encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["type", "link", "pair", "PRODUCT NUMBER", "ALCOHOL", "SUGAR", "ACIDS", "ENERGY", "PRICE / LITRE", "MORE INFORMATION", "ADDITIONAL INFORMATION", "INGREDIENTS DECLARED BY THE PRODUCER", "PACKAGING", "CLOSURE", "PRODUCER", "SUPPLIER", "SELECTION", "text", "topNotch", "tips" ])
        fileName = wine[1]
        wineType = wine[0]
        f = open(fileName+".txt", "r",encoding='utf-8')
        lines = f.readlines()
        for url in lines: 
            response = await getWine(url.strip())

            for pair in response['pairings']:
                writer.writerow([wineType,
                                url,
                                pair,
                                response['stats']["PRODUCT NUMBER"] if "PRODUCT NUMBER" in response['stats'] else "",
                                response['stats']["ALCOHOL"] if "ALCOHOL" in response['stats'] else "",
                                response['stats']["SUGAR"] if "SUGAR" in response['stats'] else "",
                                response['stats']["ACIDS"] if "ACIDS" in response['stats'] else "",
                                response['stats']["ENERGY"] if "ENERGY" in response['stats'] else "",
                                response['stats']["PRICE / LITRE"] if "PRICE / LITRE" in response['stats'] else "",
                                response['stats']["MORE INFORMATION"] if "MORE INFORMATION" in response['stats'] else "",
                                response['stats']["ADDITIONAL INFORMATION"] if "ADDITIONAL INFORMATION" in response['stats'] else "",
                                response['stats']["INGREDIENTS DECLARED BY THE PRODUCER"] if "INGREDIENTS DECLARED BY THE PRODUCER" in response['stats'] else "",
                                response['stats']["PACKAGING"] if "PACKAGING" in response['stats'] else "",
                                response['stats']["CLOSURE"] if "CLOSURE" in response['stats'] else "",
                                response['stats']["PRODUCER"] if "PRODUCER" in response['stats'] else "",
                                response['stats']["SUPPLIER"] if "SUPPLIER" in response['stats'] else "",
                                response['stats']["SELECTION"] if "SELECTION" in response['stats'] else "",
                                response['text'],
                                response['topNotch'],
                                response['tips']                             
                                
                                ])
        f.close()
    print("DONE WINE "+wine[0])
            


Starting wine rose
DONE WINE rose
Starting wine red
DONE WINE red
Starting wine sparkling
DONE WINE sparkling
Starting wine white
DONE WINE white


In [11]:
df1 = pd.read_csv("dataset_box.csv")
df2 = pd.read_csv("dataset_desert.csv")
df3 = pd.read_csv("dataset_red.csv")
df4 = pd.read_csv("dataset_rose.csv")
df5 = pd.read_csv("dataset_sparkling.csv")
df6 = pd.read_csv("dataset_white.csv")

In [13]:
frames = [df1, df2, df3, df4, df5, df6]

result = pd.concat(frames)

In [15]:
result.to_csv("dataset_all.csv")

In [28]:
result['topNotch']

0       Full-bodied, medium tannic, blackberry notes, ...
1       Full-bodied, medium tannic, blackberry notes, ...
2       Full-bodied, medium tannic, blackberry notes, ...
3       Full-bodied, medium tannic, blackberry notes, ...
4       Medium sweet, medium acidic, honey melon notes...
                              ...                        
9031    Medium sweet, medium acidic, tropical fruit no...
9032    Medium sweet, medium acidic, tropical fruit no...
9033    Medium sweet, medium acidic, tropical fruit no...
9034    Medium sweet, medium acidic, tropical fruit no...
9035    Medium sweet, medium acidic, tropical fruit no...
Name: topNotch, Length: 30252, dtype: object

In [55]:
def fuc(row):
    notches = row.split(',')
    for n in notches:
        flavorDic[n.strip()] = 0 

    flavorDicF[notches[0]] = 0

x = result['topNotch']
flavorDic = dict()
flavorDicF = dict()
a = x.apply(lambda row : fuc(row) )

In [56]:
for key in flavorDic.keys():
  if "acid"  in key:
    print(key)

medium acidic
acidic
very acidic
slightly acidic


In [57]:
for key in flavorDicF.keys():
    print(key)

Full-bodied
Medium sweet
Dry
Medium-bodied
Medium dry
Sweet
Light-bodied
Yellowy brown
Cinnamon-brown
Dark red
Amber-brown
Red
Orange
Brown
Richly red
Reddish brown
Bright red
Chestnut brown
Mahogany-brown
Copper-red
Pale
Pale yellow
Brick-red
Brownish-red
Golden-yellow
Amber-yellow
Golden brown
Copper-brown
Orange-yellow

Pink
Coffee-brown
Sweet 
Light greenish yellow
Amber-orange
Violet
Brut
Sorrel notes
Green-yellow
Purple
Light brown
Copper-yellow
Nut-brown
Brownish-yellow
Extra full-bodied
Yellow
Reddish
Dark brown
Medium full bodied
Bright yellow
Reddish violet
Very dry


In [7]:
def getLinks(fileName, url):
    option = Options()

    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")

    # Pass the argument 1 to allow and 2 to block
    option.add_experimental_option("prefs", { 
        "profile.default_content_setting_values.notifications": 1 
    })

    with webdriver.Chrome(chrome_options=option,executable_path=r"C:\Users\ramang\Developer\NN-zadania\Zadanie2\dataset-creating\chromedriver_win32\chromedriver.exe") as driver:
        wait = WebDriverWait(driver, 30)
        driver.get(url)   
        SCROLL_PAUSE_TIME = 1

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

    #        searchResult =  driver.find_elements(By.CLASS_NAME , "row product-list")


            if new_height == last_height:
                break
            last_height = new_height
        searchResult =  driver.find_elements(By.CLASS_NAME , "mini-card")
        print(len(searchResult))
        f= open(fileName+str(len(searchResult))+".txt","w+")
        for result in searchResult:
            for x in result.find_elements_by_class_name("js-product-link"):
                f.write(x.get_attribute('href')+" \n")
        f.close()

In [8]:
# ROSE WINES 
#https://www.alko.fi/en/products/ros%C3%A9-wine
#getLinks("rose", "https://www.alko.fi/en/products/ros%C3%A9-wine")

# WHITE WINES
#getLinks("white", "https://www.alko.fi/en/products/white-wine")

# RED WINES
#getLinks("red", "https://www.alko.fi/en/products/red-wine")

# CHAMPANE - Sparkling wines
#https://www.alko.fi/en/products/sparkling-wine-and-champagne
#getLinks("sparkling", "https://www.alko.fi/en/products/sparkling-wine-and-champagne")

# DESERT wines
# https://www.alko.fi/en/products/dessert-wine-and-other-fortified-wines
#getLinks("desert", "https://www.alko.fi/en/products/dessert-wine-and-other-fortified-wines")
# BAG
# https://www.alko.fi/en/products/bag-in-box-wines
#getLinks("box", "https://www.alko.fi/en/products/bag-in-box-wines")